In [1]:
just_checking_integrity=False

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%ls

drive/  sample_data/


In [5]:
items_features = pd.read_csv('drive/MyDrive/recsys2022-main/dataset/processed_data/simplified_features_and_categories_30.csv')
items_features.head()

,item_id,feature_idx
0,19021,0
1,19021,22
2,19021,15
3,19021,446
4,19021,1


In [6]:
items_features['item_id'].max()

23691

In [7]:
dataset = np.zeros(
    (items_features['item_id'].max()+1, items_features['feature_idx'].max()+1)
    )
dataset[(items_features['item_id'], items_features['feature_idx'])] = 1

dataset
train_dataset=dataset[1:,:]

In [8]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(train_dataset,test_size=0.2,random_state=941116)

In [9]:
dataset.shape

(23692, 636)

In [10]:
train.shape

(18952, 636)

In [11]:
test.shape

(4739, 636)

# Autoencoder

In [12]:
latent_dimension = 64
starting_dimension = 512

inputs = layers.Input(int(items_features['feature_idx'].max()+1))

x = inputs
dimension = starting_dimension

while(dimension != latent_dimension):
    #x = layers.BatchNormalization()(x)
    x = layers.Dense(units=dimension, activation='gelu',activity_regularizer=tf.keras.regularizers.L1L2(l1=1e-5,l2=1e-5))(x)
    x = layers.Dropout(0.1)(x)

    dimension = dimension//2

x = layers.Dense(units=dimension,activation="tanh",activity_regularizer=tf.keras.regularizers.L1L2(l1=5e-5,l2=4e-4))(x)
outputs = x
encoder = tf.keras.Model(inputs=inputs, outputs=outputs, name='encoder')

In [13]:
max_width = 1024
ending_dimension = items_features['feature_idx'].max()+1

inputs = layers.Input(latent_dimension)

x = inputs
dimension = latent_dimension

x = layers.Dense(units=256, activation='gelu',activity_regularizer=tf.keras.regularizers.L1L2(l1=1e-5,l2=1e-5))(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(units=512, activation='gelu',activity_regularizer=tf.keras.regularizers.L1L2(l1=1e-5,l2=1e-5))(x)
x = layers.Dropout(0.2)(x)


x = layers.Dense(units=ending_dimension,activation="sigmoid")(x)
outputs = x

decoder = tf.keras.Model(inputs=inputs, outputs=outputs, name='decoder')

#decoder.summary()

In [14]:
ae = tf.keras.Model(inputs=encoder.inputs, outputs=decoder(encoder.outputs), name='auto_encoder')
ae.summary()

Model: "auto_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 636)]             0         
                                                                 
 dense (Dense)               (None, 512)               326144    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)              

In [15]:
def custom_loss(x_true,x_pred):
    return tf.keras.losses.cosine_similarity(x_true,x_pred)+tf.keras.losses.mean_squared_error(x_true,x_pred)#

In [16]:
ae.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.006),
    loss=custom_loss,
    metrics=[tf.keras.metrics.CosineSimilarity(),tf.keras.metrics.MeanAbsoluteError()]
    )

# Training

In [17]:
ae.fit(
    train,
    train,
    batch_size=64,
    validation_data=(test,test),
    epochs=5 if just_checking_integrity else 200,
    shuffle=True,
    callbacks=[
            tf.keras.callbacks.EarlyStopping(
                monitor='cosine_similarity',
                patience=10,
                restore_best_weights=True,
                mode="max"
            ),
            tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_cosine_similarity',
                factor=0.7,
                patience=2,
                min_lr=5e-5,
                mode="max",
                min_delta=0.0001,
            )
        ]
    )

Epoch 1/200
297/297 [==============================] - 13s 31ms/step - loss: -0.7605 - cosine_similarity: 0.7972 - mean_absolute_error: 0.0491 - val_loss: -0.8449 - val_cosine_similarity: 0.8692 - val_mean_absolute_error: 0.0310 - lr: 0.0060
Epoch 2/200
297/297 [==============================] - 8s 28ms/step - loss: -0.8513 - cosine_similarity: 0.8745 - mean_absolute_error: 0.0305 - val_loss: -0.8722 - val_cosine_similarity: 0.8927 - val_mean_absolute_error: 0.0273 - lr: 0.0060
Epoch 3/200
297/297 [==============================] - 7s 23ms/step - loss: -0.8710 - cosine_similarity: 0.8915 - mean_absolute_error: 0.0268 - val_loss: -0.8854 - val_cosine_similarity: 0.9043 - val_mean_absolute_error: 0.0243 - lr: 0.0060
Epoch 4/200
297/297 [==============================] - 8s 28ms/step - loss: -0.8837 - cosine_similarity: 0.9027 - mean_absolute_error: 0.0243 - val_loss: -0.9001 - val_cosine_similarity: 0.9171 - val_mean_absolute_error: 0.0213 - lr: 0.0060
Epoch 5/200
297/297 [==============

In [18]:
temp=encoder(dataset)

In [19]:
temp.shape

TensorShape([23692, 64])

In [20]:
temp

<tf.Tensor: shape=(23692, 64), dtype=float32, numpy=
array([[-0.07283564, -0.17190252, -0.01174387, ...,  0.02982054,
        -0.04153243, -0.04976853],
       [ 0.17174426,  0.03563323, -0.1364352 , ...,  0.18223514,
        -0.05094333,  0.0118846 ],
       [ 0.10672751, -0.02232261, -0.10154428, ...,  0.20324971,
        -0.04768061,  0.06935072],
       ...,
       [-0.04016071, -0.06435574, -0.11192699, ..., -0.05860802,
        -0.09924958,  0.0749606 ],
       [ 0.18486916,  0.07789161, -0.05698241, ...,  0.0668982 ,
         0.04312735,  0.08086981],
       [ 0.05442367,  0.22968593, -0.00552227, ...,  0.18857884,
         0.12647034,  0.10723782]], dtype=float32)>

In [21]:
tf.reduce_sum(temp,axis=1).numpy().max(),tf.reduce_sum(temp,axis=1).numpy().min(),tf.reduce_sum(temp,axis=1).numpy().std()

(4.1276283, -4.760041, 1.0656307)

In [22]:
np.save('drive/MyDrive/recsys2022-main/dataset/processed_data/compressed_features', encoder(dataset))